#### Business Analytics FHDW 2025
# Klassifikations- und Regressionsbäume
## Konstruktion eines Klassifikationsbaums

Für den Aufbau eines Klassifikationsbaums betrachten wir noch einmal das Beispiel der Sitzrasenmäher. Der Hersteller möchte die potentiellen Käufer klassifizieren. Wir lesen den Datensatz `RidingMowers`erneut ein und teilen ihn auf in die Eigentümer und Nichteigentümer.

Für die Baumdarstellung, falls noch nicht installiert:

In [ ]:
import sys
! conda install --yes --prefix {sys.prefix} -c conda-forge pydotplus
# Bei Bedarf auch noch conda install -c conda-forge graphviz

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
from dmba import plotDecisionTree, classificationSummary, regressionSummary

mower_df = pd.read_csv('./Daten/RidingMowers.csv')
subset_owners = mower_df[mower_df['Ownership']=='Owner']
subset_nonowners = mower_df[mower_df['Ownership']=='Nonowner']

Beide Gruppen stellen wir grafisch dar. Da wir den Plot ein paar Mal brauchen, machen wir eine Funktion `create_example_plot` daraus.

In [ ]:
def create_example_plot():
    fig, ax = plt.subplots()
    ax.scatter(subset_owners.Income, subset_owners.Lot_Size, marker='o',
           label='Eigentümer', color='C0', s=30)
    ax.scatter(subset_nonowners.Income, subset_nonowners.Lot_Size, marker='o',
           label='Nichteigentümer', color='C1', facecolors='none', s=30)

    plt.xlabel('Einkommen')
    plt.ylabel('Grundstücksgröße')
    ax.set_xlim(20, 120)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, loc=4)
    
    return ax

ax = create_example_plot()
plt.show()

Nun wollen wir diesen Prädiktorraum *rekursiv partitionieren*. D. h. wir teilen den $p$-dimensionalen Raum der Variablen $X_1,..., X_p$ auf in nicht-überlappende, multidimensionale Rechtecke. Die Prädiktorvariablen können hier kontinuierlich, binär oder ordinal sein. Die Rekursion bezieht sich darauf, aufgeteilte Rechtecke wieder weiter aufzuteilen. Das Kriterium dafür ist, Rechtecke mit möglichst homogenen Gruppen zu bilden, also Datensätzen der gleichen Klasse.

Um die Heterogenität in einem solchen Bereich $A$ zu messen, bietet sich der *Gini-Index* an: $I(A)=1-\sum\limits_{k=1}^{m}{p^2_k}$ mit $p^2_k$ dem Anteil der Datensätze der Klasse $k$ in $A$. Dieses Maß ist $0$, wenn alle Datensätze zu einer Klasse gehören und $(m-1)/m$, wenn alle $m$ Klassen zu gleichen Anteilen enthalten sind, die Heterogenität also maximal ist.

In unserem Beispiel besitzt die Ausgangsdatenmenge gleiche Anteile von Eigentümern und Nichteigentümern, der Gini-Index ist also maximal:

In [ ]:
def gini_index(data, k_name):
    return 1-np.sum([(len(data[data[k_name]==k])/len(data))**2 for k in set(data[k_name])])

print('Gini-Index der Ausgangsdatenmenge = {}'.format(gini_index(mower_df, 'Ownership')))

Ein Konstruktionsalgorithmus für Klassifikationsbäume betrachtet alle Prädiktorvariablen, hier *Grundstücksgröße* und *Einkommen*, und alle möglichen Aufteilungspunkte, die einfach die Mittelpunkte zwischen zwei aufeinander folgenden Werten sind. Für jeden Aufteilungspunkt wird der Heterogenitätsgrad ermittelt und daraus ein Ranking erzeugt. Der Punkt mit der höchsten Heterogenitätsreduktion wird gewählt.

Im gegebenen Beispiel findet die erste Aufteilung an $59.7$ auf der Einkommensachse statt. Der Raum $(X_1, X_2)$ teilt sich also in zwei Rechtecke mit Einkommen $\ge 59.7$ und $< 59.7$.

In [ ]:
ax = create_example_plot()
ax.plot([59.7, 59.7], [ax.get_ylim()[0], ax.get_ylim()[1]], color='grey')
plt.show()

Ermitteln wir beispielhaft den Gini-Index des ersten Rechtecks, das aus einem Eigentümer und sieben Nichteigentümern besteht, sehen wir die reduzierte Heterogenität.

In [ ]:
first_rectangle = mower_df[mower_df.Income <= 59.7]
print('Gini-Index der ersten Aufteilung = {}'.format(gini_index(first_rectangle, 'Ownership')))

Der Klassifikationsbaumalgorithmus nutzt diese Aufteilung nun, um zu einem Knoten zwei Nachfolgerknoten zu erzeugen. Wir lassen den `DecisionTreeClassifier` von *scikit-learn* diesen ersten Schritt ausführen (also einen Baum der Tiefe 1 erzeugen) und stellen das Ergebnis dar. Grundlage für die Anpassung durch `fit` ist hier der gesamte Datensatz mit der Zielvariable *Eigentum/Ownership*.

In [ ]:
class_tree = DecisionTreeClassifier(random_state=0, max_depth=1)
class_tree.fit(mower_df.drop(columns=['Ownership']), mower_df['Ownership'])
print('Classes: {}'.format(', '.join(class_tree.classes_)))
plotDecisionTree(class_tree, feature_names=mower_df.columns[:2],
                 class_names=class_tree.classes_)

Im nächsten Schritt teilen wir das erste Rechteck weiter auf.

In [ ]:
ax = create_example_plot()
ax.plot([59.7, 59.7], [ax.get_ylim()[0], ax.get_ylim()[1]], color='grey')
ax.plot([ax.get_xlim()[0], 59.7], [21.4, 21.4], color='grey')
plt.show()

Nach Aufteilung auch des zweiten Rechtecks (ohne Abbildung) entsteht folgender Baum:

In [ ]:
class_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
class_tree.fit(mower_df.drop(columns=['Ownership']), mower_df['Ownership'])
plotDecisionTree(class_tree, feature_names=mower_df.columns[:2],
                 class_names=class_tree.classes_)

Die vollständige rekursive Aufteilung sieht dann so aus:

In [ ]:
ax = create_example_plot()
ax.plot([59.7, 59.7], [ax.get_ylim()[0], ax.get_ylim()[1]], color='grey')
ax.plot([ax.get_xlim()[0], 59.7], [21.4, 21.4], color='grey')
ax.plot([59.7, ax.get_xlim()[1]], [19.8, 19.8], color='grey')
ax.plot([84.75, 84.75], [ax.get_ylim()[0], 19.8], color='grey')
ax.plot([61.5, 61.5], [ax.get_ylim()[0], 19.8], color='grey')
plt.show()

In [ ]:
class_tree = DecisionTreeClassifier(random_state=0)
class_tree.fit(mower_df.drop(columns=['Ownership']), mower_df['Ownership'])
plotDecisionTree(class_tree, feature_names=mower_df.columns[:2],
                 class_names=class_tree.classes_)

## Performance eines Klassifikationsbaums

Wie wir schon bei den bisher betrachteten Ansätzen gelernt haben, reicht die bloße Anpassung unseres Modells an einen Trainingsdatensatz nicht aus, um zu (statistisch) belastbaren Ergebnissen zu kommen. Das Modell muss sich mit Validierungsdaten beweisen. Für die hier behandelten Bäume gilt das um so mehr: Mit unterschiedlichen Auswahlen von Datensätzen kann sich die Baumstruktur als recht instabil zeigen, außerdem neigen vollständig angepasste Bäume zu Überanpassung.

### Instabilität

Wir betrachten als weiteres Beispiel `UniversalBank` und die Akzeptanz eines persönlichen Darlehens im Kundenstamm. Die Daten zeigen uns die Ergebnisse einer vorhergegangenen Marketingkampagne mit einer Erfolgsquote von über 9%. Ein Klassifikationsbaum liefert nun einen plausiblen Ansatz zur Analyse der einzelnen Faktoren, die zum Erfolg und der Vergabe eines Darlehens führen und damit ein Modell des Kundenverhaltens bilden.

Zunächst konstruieren wir den vollständigen Baum.

In [ ]:
bank_df = pd.read_csv('./Daten/UniversalBank.csv')
bank_df
bank_df.drop(columns=['ID', 'ZIP Code'], inplace=True)

X = bank_df.drop(columns=['PersonalLoan'])
y = bank_df['PersonalLoan']
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

full_class_tree = DecisionTreeClassifier(random_state=1)
full_class_tree.fit(train_X, train_y)

plotDecisionTree(full_class_tree, feature_names=train_X.columns)

Im Klassifikationsbaum führen von 43 Endknoten 24 zum abgelehnten und 19 zum angenommenen Darlehen. Zunächst generieren wir die Konfusionsmatrizen für die Trainings- und Validierungsdaten.

In [ ]:
classificationSummary(train_y, full_class_tree.predict(train_X))
print()
classificationSummary(valid_y, full_class_tree.predict(valid_X))

Wie zu erwarten, werden die Trainingsdaten genau klassifiziert, die Validierungsdaten allerdings nur mit einer Genauigkeit von knapp 98%.

Im Falle des Baums können diese Ergebnisse je nach Auswahl der Datensätze stark variieren. Daher ist hier eine *Kreuzvalidierung* der plausiblere Ansatz für die Prüfung der Performance, um direkt einen Eindruck dieser Schwankungen bzw. eine Sensitivitätsanalyse gegenüber verschiedenen Datenpartitionen zu erhalten. Eine *k-fold cross-validation* partitioniert die Daten in $k$ nicht-überlappende Stichproben (*folds*; "Faltungen"). Das Modell wird dann $k$-mal angepasst. In jedem Durchlauf ist eine der Stichproben der Validierungsdatensatz, die restlichen $k-1$ Stichproben sind die Trainingsdaten. Üblich sind $k=5$ cross-validations.

In [ ]:
tree_classifier = DecisionTreeClassifier(random_state=1)
# Das Fitting macht die folgende Funktion selbst; ist hier also nicht nötig.
scores = cross_val_score(tree_classifier, train_X, train_y, cv=5)
print('Genauigkeit über jede Stichprobe: ', [f'{acc:.3f}' for acc in scores])

### Überanpassung

Bei der oben gezeigten Konstruktion eines Baums erwarten wir, dass mit jedem Wachstumsschritt die Genauigkeit der Klassifikation zunimmt. Für die Trainingsdaten trifft das bis zum maximalen Ausbau auch zu. Für neue Datensätze ist allerdings i. d. R. bereits früher ein Punkt erreicht, an dem die Beziehung zwischen Prädiktoren und Klasse vollständig (oder so vollständig wie auf Basis der Daten möglich) modelliert ist. Ab diesem Punkt beginnt der Baum, das Rauschen der Trainingsdaten zu modellieren und der Fehler nimmt wieder zu. Eine intuitive Erklärung bei sehr großen Bäumen sind die irgendwann sehr kleinen Stichproben/Samples der letzten Entscheidungsknoten.

Es gibt eine Reihe von nahe liegenden Ansätzen, das Wachstum zu begrenzen: 
- Maximale Tiefe, 
- minimale Anzahl von Datensätzen in Endknoten, 
- minimale Reduktion von Heterogenität durch einen Aufteilungsschritt. 

Zu diesen Parametern gibt es aber keine einfachen Regeln oder belastbare allgemeine Heuristiken, um ihren Wert zu bestimmen.

Da wir sie im `DecisionTreeClassifier` zur Steuerung der Modellkonstruktion nutzen können, ein Beispiel mit einem beschränkten Baum über die Trainingsdaten aus `UniversalBank`.

In [ ]:
# Mit den Parametern können Sie gut selbst experimentieren:
small_class_tree = DecisionTreeClassifier(max_depth=30, 
                                          min_samples_split=20, 
                                          min_impurity_decrease=0.01, 
                                          random_state=1)
small_class_tree.fit(train_X, train_y)

plotDecisionTree(small_class_tree, feature_names=train_X.columns)

In [ ]:
classificationSummary(train_y, small_class_tree.predict(train_X))
print()
classificationSummary(valid_y, small_class_tree.predict(valid_X))
print()
scores = cross_val_score(small_class_tree, train_X, train_y, cv=5)
print('Genauigkeit über jede Stichprobe: ', [f'{acc:.3f}' for acc in scores])

Der reduzierte Baum bietet - für neue Datensätze - eine ähnliche Qualität der Klassifikation wie der unbeschränkte Baum. 

Auch, wenn wir für die beschränkenden Parameter nicht einfach die optimalen Werte berechnen können, hilft uns auch hier wieder der Rechner durch *brute force*: Eine *Gittersuche* (*grid search*) ermittelt aus verschiedenen Wertekombinationen die, die zum Baum mit dem kleinsten Fehler führt. Damit dabei nicht eine Überanpassung an Trainings- oder Validierungsdaten erfolgt, führt die Suche zunächst *cross-validation* (s. o.) auf die Trainingsdaten aus und evaluiert dann die Performance mit den Validierungsdaten.

Eine erschöpfende Gittersuche ist implementiert in `GridSearchCV`. Sie kann schnell rechenintensiv werden. Wir fangen an mit einer initialen Schätzung der Parameter. Dieser erste Durchlauf erfordert $4 \cdot 5 \cdot 5 = 100$ Kombinationen. Bei mehr Parametern steigt der Aufwand entsprechend.

In [ ]:
param_grid = {
    'max_depth': [10, 20, 30, 40],
    'min_impurity_decrease': [0, 0.0005, 0.001, 0.005, 0.01],
    'min_samples_split': [20, 40, 60, 80, 100]
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=1), param_grid, cv=5, n_jobs=-1)
grid_search.fit(train_X, train_y)
print('Initiale Bewertung: ', grid_search.best_score_)
print('Initiale Parameter: ', grid_search.best_params_)

Die resultierenden Werte liefern uns Hinweise, in welche Richtungen eine Verfeinerung suchen sollte:

In [ ]:
param_grid = {
    'max_depth': list(range(5, 15)),
    'min_impurity_decrease': [0.0001, 0.0005, 0.001],
    'min_samples_split': list(range(10, 30))
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=1), param_grid, cv=5, n_jobs=-1)
grid_search.fit(train_X, train_y)
print('Verbesserte Bewertung: ', grid_search.best_score_)
print('Verbesserte Parameter: ', grid_search.best_params_)

best_class_tree = grid_search.best_estimator_

So bekommen wir einen weiter verbesserten Baum mit folgender Performance und Form:

In [ ]:
classificationSummary(train_y, best_class_tree.predict(train_X))

In [ ]:
classificationSummary(valid_y, best_class_tree.predict(valid_X))

In [ ]:
plotDecisionTree(best_class_tree, feature_names=train_X.columns)

## Aufgabe

Wir betrachten noch einmal den Datensatz *FlightDelays.csv*.

1. Interaktiv: Was sind hier sinnvolle Prädiktoren für die Verspätung von Flügen?

2. Bereiten Sie den Datensatz für die Generierung von Entscheidungsbäumen vor: Wandeln Sie die erforderlichen Variablen in kategorische um, erzeugen Sie Dummy-Variablen und teilen Sie 60% Trainingsdaten und 40% Validierungsdaten auf.

3. Generieren Sie einen Entscheidungsbaum mit einer maximalen Tiefe von 8 zur Vorhersage von Verspätungen und plotten Sie ihn. Lassen sich daraus einfache praktische Regeln für die Vorhersage von Verspätungen ableiten?

4. Beurteilen Sie die Qualität der Vorhersage und vergleichen Sie diese mit den Ergebnissen der naiven Bayes-Klassifikation (Konfusionsmatrizen und Kreuzvalidierung). Verändern Sie die maximale Tiefe des Baums und beurteilen Sie die Auswirkungen auf die Prädiktionen (besser, schlechter).

In [ ]:
delays_df = pd.read_csv('./Daten/FlightDelays.csv')
delays_df

## Regressionsbäume

Eine numerische Zielvariable erhalten wir mit *Regressionsbäumen*. Das Konstruktionsprinzip bleibt hier das gleiche, wie bei der Klassifikation: Der Algorithmus minimiert die Heterogenität unterschiedlicher Aufteilungen des Prädiktorraums.

Zur Veranschaulichung der Unterschiede zwischen Regressions- und Klassifikationsbäumen betrachten wir noch einmal das Beispiel der Gebrauchtwagenbewertung mit dem Datensatz `ToyotaCorolla`. Wir generieren uns mit dem oben gezeigten Vorgehen einen reduzierten Baum dazu. Dieses Mal nutzen wir aber den `DecisionTreeRegressor`. 

In [ ]:
toyota_corolla_df = pd.read_csv('./Daten/ToyotaCorolla.csv').iloc[:1000,:]
toyota_corolla_df = toyota_corolla_df.rename(columns={'age_08_04': 'age', 'quarterly_tax': 'tax'})

predictors = ['age', 'km', 'fuel_type', 'hp', 'met_color', 'automatic', 'cc', 'doors', 'tax', 'weight']
outcome = 'price'

X = pd.get_dummies(toyota_corolla_df[predictors], drop_first=True)
y = toyota_corolla_df[outcome]

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

param_grid = {
    'max_depth': [5, 10, 15, 20, 25],
    'min_impurity_decrease': [0, 0.001, 0.005, 0.01],
    'min_samples_split': [10, 20, 30, 40, 50]
}

grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(train_X, train_y)
print('Initiale Parameter: ', grid_search.best_params_)
regression_tree = grid_search.best_estimator_
regressionSummary(valid_y, regression_tree.predict(valid_X))

param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'min_impurity_decrease': [0, 0.001, 0.002, 0.003, 0.005, 0.006, 0.007, 0.008],
    'min_samples_split': [14, 15, 16, 18, 20]
}

grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(train_X, train_y)
print()
print('Verbesserte Parameter: ', grid_search.best_params_)
regression_tree = grid_search.best_estimator_
regressionSummary(valid_y, regression_tree.predict(valid_X))

plotDecisionTree(regression_tree, feature_names=train_X.columns, rotate=True)

### Prädiktion

Zunächst sehen wir in den oberen Ebenen des Baums, welche Einflussfaktoren bzw. Prädiktoren beim Preis die größte Rolle spielen. Möchten wir den Preis eines Fahrzeugs mit z. B. einem *Alter von 60, Kilometerstand von 160.000 und 100 PS* voraussagen, erreichen wir den Endknoten mit dem Wert *8392.857*. Die Klassifikation oben wurde durch die Mehrheit der Trainingsdatensätze erreicht, die im Endknoten ausgewertet wurden. Die Regression ermittelt statt dessen den numerischen Zielwert aus den Durchschnittswerten dieser Trainingsdatensätze - im Fall hier ist 8.392,86 der Durchschnittspreis der sieben Fahrzeuge aus den Trainingsdaten, die über 49.5 Monate alt sind, einen Kilometerstand über 128.361 und über 93.5 PS haben.

In [ ]:
subset_cars = train_X[(train_X['age']>=49.5) 
                      & (train_X['km']>=128361)
                      & (train_X['hp']>=93.5)].join(train_y)
node_mean = subset_cars['price'].mean()
print(node_mean)
subset_cars

### Heterogenität

Bei der Klassifikation liefert der *Gini-Index* das Verhältnis zwischen Klassen von Datensätzen in einem Knoten. Bei der Regression ist ein typisches Heterogenitätsmaß die Summe der quadrierten Abweichungen vom Mittelwert eines Endknotens. Da dieses Mittel die Prädiktion repräsentiert, entspricht das der bekannten Summe der Fehlerquadrate. Für das Beispiel oben:

In [ ]:
for _, car in subset_cars.iterrows():
    print('(Preis {:.2f} - {:.2f})^2 = {:.2f}'.format(car['price'], node_mean, (car['price']-node_mean)**2))

# Oder einfach so:
sum((subset_cars['price']-node_mean)**2)

### Performance

Wie wir oben schon bei der Konstruktion des Regressionsbaums gesehen haben, gelten für die Beurteilung der Vorhersagequalität hier die gleichen Größen wie bei anderen Regressionen, z. B. der linearen, also Fehlersummen wie RMSE.

## Verbesserung der Vorhersage: Random Forests und Boosted Trees

Die Nutzung eines einzelnen Baums trägt durch die gute Darstellbarkeit - wie wir oben gesehen haben - sehr zur Transparenz und Nachvollziehbarkeit dieser Methode bei. Wenn das jedoch zu Gunsten der Performance vernachlässigt werden kann, können wir *Ensembles* aus mehreren Bäumen bilden. Die folgenden Ansätze sind sowohl für die Klassifikation, als auch die Regression nutzbar (`Regressor` statt `Classifier` in den Bibliotheken).

### Random Forests

Das grundsätzliche Vorgehen sieht bei diesen "zufälligen Wäldern" so aus:

1. Mehrere zufällige Stichproben mit Zurücklegen aus den Daten ziehen (*bootstrap*).
2. Jede Stichprobe mit einer zufälligen Auswahl an Prädiktoren an einen Klassifikations- oder Regressionsbaum anpassen (so entsteht der Wald).
3. Aus den Vorhersagen der einzelnen Bäume die Mehrheitsentscheidungen für Klassifikation nutzen bzw. die Durchschnitte für Regression bilden. Aus diesen Kombinationen erhoffen wir uns verbesserte Vorhersagen.

Wenden wir diese Methode auf unser Bank-Szenario an (`n_estimators` ist die Anzahl der Bäume des Waldes):

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

bank_df = pd.read_csv('./Daten/UniversalBank.csv')
bank_df
bank_df.drop(columns=['ID', 'ZIP Code'], inplace=True)

X = bank_df.drop(columns=['PersonalLoan'])
y = bank_df['PersonalLoan']
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

random_forest = RandomForestClassifier(n_estimators=500, random_state=1)
random_forest.fit(train_X, train_y)

Wir können hier keine, oder zumindest keine übersichtlichen, Baumdiagramme mehr zur Visualisierung nutzen. Die Wälder liefern aber Bewertungen der Wichtigkeit einzelner Variablen: Das ist hier ihr relativer Beitrag in Form der summierten Abnahme des Gini-Index für diesen Prädiktor über alle Bäume im Wald.

Für unser Beispiel:

In [ ]:
importances = random_forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in random_forest.estimators_], axis=0)

df = pd.DataFrame({'Eigenschaft': train_X.columns, 'Wichtigkeit': importances, 'Std': std})
df = df.sort_values('Wichtigkeit', ascending=False)
df

In [ ]:
df = df.sort_values('Wichtigkeit')
ax = df.plot(kind='barh', xerr='Std', x='Eigenschaft', legend=False)
ax.set_ylabel('')
plt.show()

Der *Random Forest* liefert uns für das gegebene Szenario eine ähnliche Genauigkeit der Vorhersage der Validierungsdaten wie die *Gittersuche* oben:

In [ ]:
classificationSummary(valid_y, random_forest.predict(valid_X))

### Boosted Trees

Eine weitere Möglichkeit, mehrere Bäume für die Vorhersage zu nutzen, liegt in einer Sequenz aus Bäumen, in der jeder Folgebaum die Fehlklassifikationen des Vorgängers kompensieren soll:

1. Anpassen eines einzelnen Baumes.
2. Dazu eine Stichprobe finden, in der die Wahrscheinlichkeit für fehlklassifizierte Datensätze hoch ist.
3. Einen Folgebaum auf diese Stichprobe anpassen.
4. Die Schritte 2 und 3 mehrfach wiederholen.
5. Aus den gewichteten Ergebnissen die Vorhersagen generieren, mit größerem Gewicht auf hintere Bäume in der Sequenz.

Wenden wir auch diesen Ansatz auf unser Beispiel an:

In [ ]:
boosted = GradientBoostingClassifier()
boosted.fit(train_X, train_y)
classificationSummary(valid_y, boosted.predict(valid_X))

Wir erhalten eine weiter leicht verbesserte Genauigkeit der Vorhersage. 

Insbesondere ist aber der Fehler in der Klassifikation der *1* reduziert. Dies ist eine besondere Eigenschaft der *Boosted Trees*: Die einfachen Klassifizierer lassen sich stark von dominierenden Klassen beeinflussen (*0* macht im gegebenen Datensatz über 90% aus). Entsprechend gibt es relativ viele Fehlklassifikationen der *1* in einem einzelnen Baum. Der oben kurz beschriebene Boosting-Algorithmus fokussiert sich aber genau auf diese Fehlklassifikationen und hat daher gute Chancen, sie zu reduzieren.

## Aufgabe

Versuchen Sie, mit *Random Forests* und *Boosted Trees* die Genauigkeit des Vorhersagemodells für die Flugverspätungen aus der letzten Aufgabe zu verbessern.